In [1]:
!pwd

/Users/apple/Desktop/Desktop/AIML/TextMiningPipeline/ClauseClassifier/research


In [2]:
import os
os.chdir("../")

In [3]:
!pwd

/Users/apple/Desktop/Desktop/AIML/TextMiningPipeline/ClauseClassifier


In [4]:
## Entity Return type of a function 
from dataclasses import dataclass 
from pathlib import Path


@dataclass(frozen=True)
class DataIngetionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [5]:
from clause_classifier.constants import *
from clause_classifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngetionConfig:
        config = self.config.data_ingestion 
        create_directories([config.root_dir])

        data_ingestion_config = DataIngetionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )
        return data_ingestion_config


In [7]:
## components 
import os 
import urllib.request as request 
import zipfile 
from clause_classifier.logging import logger 
from clause_classifier.utils.common import get_size 

In [8]:
class DataIngestion: 
    def __init__(self, config: DataIngetionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, hearders = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info:\n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir 
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Files extracted to: {unzip_path}")

In [9]:
## create pipeline 
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error occurred during data ingestion: {e}")

[2024-11-02 23:09:51,687]:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-11-02 23:09:51,690]:INFO:common:yaml file: params.yaml loaded successfully]
[2024-11-02 23:09:51,691]:ERROR:652317423:Error occurred during data ingestion: 'exist_ok' is an invalid keyword argument for mkdir()]
